# Ekşi Web Scraping - Topics

In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\melihcanyardi\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [2]:
eksi_gundem = pd.read_csv("eksi_gundem.csv")
eksi_gundem = eksi_gundem.drop_duplicates("Topics").reset_index(drop=True)
eksi_gundem.head()

,Topics,URL,Entry_Counts,Snapshot_Time
0,exxen,https://eksisozluk.com/exxen--6679701,532,2021-01-02 04:05:42
1,akıncı (dizi),https://eksisozluk.com/akinci-dizi--6747980,323,2021-01-02 04:05:42
2,rte'nin başörtülü kadına vitrin mankeni demesi,https://eksisozluk.com/rtenin-basortulu-kadina...,225,2021-01-02 04:05:42
3,insanlık tarihinin en büyük buluşu,https://eksisozluk.com/insanlik-tarihinin-en-b...,246,2021-01-02 04:05:42
4,yazarların 2020 ve 2021 başı kilo durumları,https://eksisozluk.com/yazarlarin-2020-ve-2021...,529,2021-01-02 04:05:42


In [3]:
#eksi_gundem_jan = eksi_gundem[(eksi_gundem["Snapshot_Time"] >= "2021-01-01") & (eksi_gundem["Snapshot_Time"] < "2021-02-01")].reset_index(drop=True)
#eksi_gundem_feb = eksi_gundem[(eksi_gundem["Snapshot_Time"] > "2021-02-01") & (eksi_gundem["Snapshot_Time"] < "2021-03-01")].reset_index(drop=True)
#eksi_gundem_mar = eksi_gundem[(eksi_gundem["Snapshot_Time"] > "2021-03-01") & (eksi_gundem["Snapshot_Time"] < "2021-04-01")].reset_index(drop=True)
#eksi_gundem_apr = eksi_gundem[(eksi_gundem["Snapshot_Time"] > "2021-04-01") & (eksi_gundem["Snapshot_Time"] < "2021-05-01")].reset_index(drop=True)
#eksi_gundem_may = eksi_gundem[(eksi_gundem["Snapshot_Time"] > "2021-05-01") & (eksi_gundem["Snapshot_Time"] < "2021-06-01")].reset_index(drop=True)
#eksi_gundem_jun = eksi_gundem[(eksi_gundem["Snapshot_Time"] > "2021-06-01") & (eksi_gundem["Snapshot_Time"] < "2021-07-01")].reset_index(drop=True)
#eksi_gundem_jul = eksi_gundem[(eksi_gundem["Snapshot_Time"] > "2021-07-01") & (eksi_gundem["Snapshot_Time"] < "2021-08-01")].reset_index(drop=True)
#eksi_gundem_aug = eksi_gundem[(eksi_gundem["Snapshot_Time"] > "2021-08-01") & (eksi_gundem["Snapshot_Time"] < "2021-09-01")].reset_index(drop=True)
#eksi_gundem_sep = eksi_gundem[(eksi_gundem["Snapshot_Time"] > "2021-09-01") & (eksi_gundem["Snapshot_Time"] < "2021-10-01")].reset_index(drop=True)
#eksi_gundem_oct = eksi_gundem[(eksi_gundem["Snapshot_Time"] > "2021-10-01") & (eksi_gundem["Snapshot_Time"] < "2021-11-01")].reset_index(drop=True)
eksi_gundem_nov = eksi_gundem[(eksi_gundem["Snapshot_Time"] > "2021-11-01") & (eksi_gundem["Snapshot_Time"] < "2021-12-01")].reset_index(drop=True)
eksi_gundem_dec = eksi_gundem[(eksi_gundem["Snapshot_Time"] > "2021-12-01") & (eksi_gundem["Snapshot_Time"] < "2022-01-01")].reset_index(drop=True)

In [4]:
def eksi_scrap(url):
    url = url
    driver.get(url)
    
    entry_item_list = driver.find_elements_by_xpath('//*[@id="entry-item-list"]')[0]
    
    entries = []
    authors = []
    dates = []
    
    for i in range(len(entry_item_list.find_elements_by_class_name("content"))):
        entries.append(entry_item_list.find_elements_by_class_name("content")[i].text)
        authors.append(entry_item_list.find_elements_by_class_name("entry-author")[i].text)
        dates.append(entry_item_list.find_elements_by_class_name("entry-date")[i].text)
        
    df = pd.DataFrame(columns = ["Entry", "Author", "Date"])
    
    entries = pd.Series(entries, name='Entry')
    authors = pd.Series(authors, name='Author')
    dates = pd.Series(dates, name='Date')
    
    df = df.append(pd.concat([entries, authors, dates], axis=1))
    return df

In [5]:
def get_entries(url):
    print(f"Getting Entries in: {url}")
    driver.get(url)
    
    # Check if the topic was changed to a new one
    try:
        if driver.find_element_by_xpath('//*[@id="title"]/sup/a'):
            old_topic = driver.find_element_by_xpath('//*[@id="title"]/sup/a').get_attribute("title")[1:-15]
    except:
        old_topic = driver.find_element_by_xpath('//*[@id="title"]/a/span').text
    
    if driver.find_element_by_xpath('//*[@id="entry-author"]/a').text == 'başlık taşıyan':
        url = driver.find_elements_by_xpath('//*[@id="entry-item-list"]')[0].find_element_by_class_name("content").find_element_by_class_name("b").get_attribute("href")
        driver.get(url)
        
    url = driver.current_url
    driver.get(url)
        
    if "true&rf" in url:
        url = driver.find_element_by_xpath('//*[@id="title"]/a').get_attribute("href")
        driver.get(url)
    
    new_topic = driver.find_element_by_xpath('//*[@id="title"]/a/span').text
    
    # Check if the topic has more than one page
    try:
        total_page = driver.find_element_by_xpath('//*[@id="topic"]/div[1]/div[2]/a[1]').text
    except:
        total_page = 1
    
    url = url + "?p="
    
    df = pd.DataFrame(columns = ["Entry", "Author", "Date"])
    
    i = 1
    while True:
        try:
            df = df.append(eksi_scrap(url + str(i)))
            print(f"Page: {i}|{total_page}")
            i += 1
        except:
            break
            
    df = df.reset_index(drop=True)
    old_topics = pd.Series([old_topic]*df.shape[0], name="Old_Topic")
    new_topics = pd.Series([new_topic]*df.shape[0], name="New_Topic")
    
    df = pd.concat([df, old_topics, new_topics], axis=1)
    return df

## November Entries

In [6]:
entries_nov = pd.DataFrame(columns = ["Entry", "Author", "Date"])

In [7]:
for i, url in enumerate(eksi_gundem_nov["URL"]):
    print(f"Topic: {eksi_gundem_nov.loc[i, 'Topics']} (#{i+1}|{eksi_gundem_nov.shape[0]})")
    try:
        entries_nov = entries_nov.append(get_entries(url))
    except:
        entries_nov = entries_nov.append(get_entries(url))

In [9]:
entries_nov = entries_nov.reset_index(drop=True)

In [10]:
## If the entry is edited, Edit is 1, else it is 0.
entries_nov['Edited'] = entries_nov['Date'].apply(lambda x: True if '~' in x else False)

# Edit Date
entries_nov['Entry_Date'] = pd.to_datetime(entries_nov['Date'].apply(lambda x: x.split('~')[0][:-1] if '~' in x else x))
entries_nov['Edit_Date'] = pd.to_datetime(entries_nov['Date'].apply(lambda x: x.split('~')[1][1:] if '~' in x else None))

In [11]:
entries_nov.to_csv("entries_nov.csv", index=False)

## December Entries

In [12]:
entries_dec = pd.DataFrame(columns = ["Entry", "Author", "Date"])

In [62]:
for i, url in enumerate(eksi_gundem_dec["URL"]):
    print(f"Topic: {eksi_gundem_dec.loc[i, 'Topics']} (#{i+1}|{eksi_gundem_dec.shape[0]})")
    try:
        entries_dec = entries_dec.append(get_entries(url))
    except:
        entries_dec = entries_dec.append(get_entries(url))

In [34]:
entries_dec = entries_dec.reset_index(drop=True)

In [36]:
## If the entry is edited, Edit is 1, else it is 0.
entries_dec['Edited'] = entries_dec['Date'].apply(lambda x: True if '~' in x else False)

# Edit Date
entries_dec['Entry_Date'] = pd.to_datetime(entries_dec['Date'].apply(lambda x: x.split('~')[0][:-1] if '~' in x else x))
entries_dec['Edit_Date'] = pd.to_datetime(entries_dec['Date'].apply(lambda x: x.split('~')[1][1:] if '~' in x else None))

In [38]:
entries_dec.to_csv("entries_dec.csv", index=False)

## Combining Entries

In [46]:
entries_nov = pd.read_csv("entries_nov.csv", parse_dates=[6, 7])
entries_dec = pd.read_csv("entries_dec.csv", parse_dates=[6, 7])

In [53]:
entries_all = pd.concat([entries_nov, entries_dec]).reset_index(drop=True)

In [59]:
entries_all.to_csv("entries_all.csv", index=False)

In [60]:
entries_nov_dec = entries_all[(entries_all["Entry_Date"] >= "2021-11-01") & (entries_all["Entry_Date"] < "2022-01-01")]["Entry_Date"]

In [61]:
entries_nov_dec.to_csv("entries_nov_dec.csv", index=False)